# 4 Classes & OOP

## 4.1 Object comparisons: 'is' vs '=='

### `is` tests if two variables point to the same object.

In [1]:
t1 = (2,)
t2 = (2,)

t1 is t2

False

In [2]:
t2 = t1
t1 is t2

True

### `==` test if two variables point to objects with equal contents.

In [3]:
t1 = (2,)
t2 = (2,)

t1 == t2

True

## 4.2 String Conversion (every class need a `__repr__`)

### `__str__` vs `__repr__`

`__str__` should be readable. `__repr__` should be unambiguous and informative for developers.

In [4]:
class MyAdder():
    def __init__(self, num):
        self.num = num
    
    def __str__(self):
        return 'MyAdder __str__'
        
    def __repr__(self):
        return 'MyAdder __repr__'

In [5]:
a = MyAdder(1)
print(a)

MyAdder __str__


In [6]:
a

MyAdder __repr__

In [7]:
str([a])

'[MyAdder __repr__]'

### Always add `__repr__` since this is fallback for `__str__`

In [8]:
class MyAdder():
    def __init__(self, num):
        self.num = num
        
    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'{self.num!r})')  # !r to use __repr__ of self.num

In [9]:
a = MyAdder(1)
print(a)

MyAdder(1)


In [10]:
repr(a)

'MyAdder(1)'

In [11]:
a

MyAdder(1)

## 4.3 Defining your own exception classes

In [12]:
class InvalidNumberError(ValueError):
    pass

class DivisionByZeroError(InvalidNumberError):
    pass

class NegativeNumberError(InvalidNumberError):
    pass

def divide(a, b):
    if b == 0:
        raise DivisionByZeroError()
    if b < 0:
        raise NegativeNumberError()
    return a/b

divide(1, 0)

DivisionByZeroError: 

- eases debugging by making errors more informative
- package specific exception hierarchy allows to catch specific types of errors only

```python
try:
    # something
except InvalidNumberError as err:
    # handle err
```

## 4.4 Cloning Objects for Fun and Profit

### Shallow vs deep copying

In [13]:
l = [[1, 2, 3]]
ll = list(l)  # shallow copy
ll[0][0] = 0
l[0]

[0, 2, 3]

In [14]:
import copy

l = [[1, 2, 3]]
ll = copy.deepcopy(l)  # deep copy
# ll = copy.copy(l)  # good for explicitely making a shallow copy
ll[0][0] = 0
l[0]

[1, 2, 3]

### Copying arbitrary objects

In [15]:
class Car:
    def __init__(self, color, wheels):
        self.color = color
        self.wheels = wheels
    
    def __repr__(self):
        return f'Car ({self.color!r}, {self.wheels!r})'
        
c = Car('blue', ['A', 'B', 'C', 'D'])
cc = copy.copy(c)  # shalow copy
print(cc)
c is cc

Car ('blue', ['A', 'B', 'C', 'D'])


False

In [16]:
c.wheels is cc.wheels

True

In [17]:
cc = copy.deepcopy(c)  # shalow copy
c is cc

False

In [18]:
c.wheels is cc.wheels

False

Implementing `__copy__()` and `__deepcopy__()` for a class allows to change how objects are copied.

## 4.5 Abstract Base Classes keep inheritance in check

Use `abc` module to avoid instantiation of abstract classes.

In [19]:
from abc import ABCMeta, abstractmethod

class Vehicle(metaclass=ABCMeta):
    @abstractmethod
    def drive(self):
        pass
    
    def honk(self):
        print('!')

In [20]:
v = Vehicle()

TypeError: Can't instantiate abstract class Vehicle with abstract methods drive

In [21]:
class Car(Vehicle):
    pass

c = Car()

TypeError: Can't instantiate abstract class Car with abstract methods drive

In [22]:
class Car(Vehicle):
    def drive(self):
        print('ROAR')

c = Car()
c.honk()
c.drive()

!
ROAR


## 4.6 What namedtuples are good for

Enable easy definition of immutable classes. Use namedtuples to introduce structure into code over, e.g., dicts or plain tuples. Don't overuse.

Declaration by passing a class name and field names (automatically split at whitespace):

In [24]:
from collections import namedtuple

Car = namedtuple('Car', 'color wheels')
c = Car('blue', [1, 2, 3, 4])
c

Car(color='blue', wheels=[1, 2, 3, 4])

In [25]:
c.color

'blue'

In [26]:
c.color = 'red'  # it's still immutable

AttributeError: can't set attribute

### Subclassing namedtuples

In [27]:
class ThreeWheeledCar(Car):
    def wheels(self):
        return [1, 2, 3]

t = ThreeWheeledCar('blue', [1, 2, 3, 4])
t.wheels()

[1, 2, 3]

### Built-in methods (start with underscore to avoid clashes with field names)

In [28]:
c._asdict()

OrderedDict([('color', 'blue'), ('wheels', [1, 2, 3, 4])])

## 4.7 Class vs Instance Variable Pitfalls

Class variables are shared across all instances of class; instance variables are unique to instance.

In [42]:
class Car:
    wheels = 4  # class variable
    car_instances = 0  # class variable (this one is actually useful)
    
    def __init__(self, color):
        self.color = color  # instance variable
        self.__class__.car_instances += 1  # note the extra __class__
        
a = Car('red')
b = Car('blue')

print('Cars existing:', Car.car_instances)

a.color = 'orange'
print(a.color, b.color)

a.wheels = 3
print(a.wheels, b.wheels, Car.wheels, a.__class__.wheels)

Cars existing: 2
orange blue
3 4 4 4


**Careful**: this created a new instance variable `a.wheels`.

## 4.8 Instance, Class, and Static Methods